In [2]:
import pandas as pd
import re
import math

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [6]:
flower_data = pd.read_csv("../Data/General/flower_v2.csv", error_bad_lines=False, delimiter=";")
flower_data = flower_data[flower_data.isna().Flower == False]
flower_data.reset_index(drop=True, inplace=True)
#flower_data.to_csv("../Data/General/flower_v2.csv", sep=";", index = False)
flower_data

,Flower,Color,Meaning
0,Acacia,NaN,Secret love
1,Acanthus,NaN,Art
2,Aconite,NaN,Misanthropy
3,Agrimony,NaN,Thankfulness
4,Aloe,NaN,Grief
5,Almond,NaN,Promise
6,Amaranth (Globe),NaN,Immortal love
7,Amaranth,NaN,Immortality [1]
8,Amaryllis,NaN,Pride
9,Ambrosia,NaN,Love is reciprocated


## Data cleaning

- Lowercase all the descriptions
- Delete the reference [*] to text
- Separate entries for each flower color: flower + [color]
- Separate entries for each meaning: separate by their ','

#### 1. Lowercase all the descriptions and delete [*] references

In [25]:
flower_data['Meaning'] = flower_data['Meaning'].str.lower().str.replace("\[.*\]", "").str.rstrip()
flower_data['Meaning'].sample(10)

211                     goodbye
145      absence, bitter sorrow
161                       pious
65                   insecurity
68     gentility, determination
174                       lucky
189                    faithful
81          return my affection
206                   desperate
138                faithfulness
Name: Meaning, dtype: object

#### 2. Fill in flower name for each flower color entries: flower + [color]

In [26]:
type(flower_data['Flower'].iat[143]) == str

True

In [27]:
precedent_flower = ""
for i in range(0, len(flower_data["Flower"])):
    
    if(type(flower_data["Flower"].iat[i]) == str):
        precedent_flower = flower_data["Flower"].iat[i]
        if(type(flower_data["Color"].iat[i]) == str):
            flower_data["Flower"].iat[i] = precedent_flower + " [" + flower_data["Color"].iat[i] + "]" 
   
    if(type(flower_data["Flower"].iat[i]) == float):
        if(type(flower_data["Color"].iat[i]) == str):
            flower_data["Flower"].iat[i] = precedent_flower + " [" + flower_data["Color"].iat[i] + "]" 
        else: 
            flower_data["Flower"].iat[i] = precedent_flower

flower_data.tail(50)

,Flower,Color,Meaning
167,Camellia (Yellow),NaN,longing
168,Camellia (White),NaN,waiting
169,Fritillaria camschatcensis,NaN,"love, curse"
170,Carnation,NaN,"fascination, distinction, and love"
171,Cherry Blossom,NaN,kind/gentle/transience of life
172,Chrysanthemum (Yellow),NaN,imperial
173,Chrysanthemum (White),NaN,truth
174,Four-leaf clover,NaN,lucky
175,Daffodil,NaN,respect
176,Dahlia,NaN,good taste


In [28]:
flower_data_split_meaning = pd.DataFrame(flower_data.Meaning.str.replace(";", ",")
                                                            .str.replace(" or ", ",")
                                                            .str.replace("/", ",")
                                                            .str.split(',').values.tolist(), index=flower_data.Flower).stack()


flower_data_split_meaning = flower_data_split_meaning.reset_index()[[0, 'Flower']]
flower_data_split_meaning.columns = ['Meaning', 'Flower'] 
flower_data_clean = flower_data_split_meaning[['Flower', 'Meaning']]
flower_data_clean.sample(20)

,Flower,Meaning
277,Calendula/Marigold,lovable
186,Oats,music
350,Poppy (Red),fun-loving
55,Bulrush,docility
132,Heather [purple],solitude
361,Rose (Pink),happiness
104,Dandelion,overcoming hardship
64,Carnation [general],love
83,Crocus,lupercalia
351,Poppy (White),rejoice


### Export the clean data to csv

In [29]:
flower_data_clean.to_csv("../Data/General/flower_cleaned.csv", sep=";", index = False)